In [1]:
import datetime as dt
import pandas as pd
import sqlite3

conexion = sqlite3.connect("./piezas.db")

In [2]:
def consulta_elemento(tabla, nombre):
    query = f"""
        SELECT *
        FROM {tabla}
        WHERE Nombre LIKE '%{nombre}%'
        """

    ID = pd.read_sql(query, conexion).ID

    if len(ID) == 0:
        return 'No se encontró ID.'
    return ID

def consulta_id(tabla, ID:list):
    query = f"""
        SELECT *
        FROM {tabla}
        WHERE ID = '{ID}'
        """

    return pd.read_sql(query, conexion)['Nombre'][0]

In [ ]:
hoy = dt.datetime.now()

for tabla in ['Proveedores','Pieza']:
    algo = 'proveedor' if tabla == 'Proveedores' else 'pieza'

    elemento = input(f'Inserte {algo}:').capitalize()
    resultados = consulta_elemento(tabla, elemento)

    if type(resultados) == str:
        print(resultados)
    else:
        if len(resultados) > 1:
            for index, resultado in enumerate(resultados):
                print(index, consulta_id(tabla, resultado))

            ID = input(f'Seleccione {algo}:')
            while ID.isnumeric() != True or int(ID) > len(resultados)-1 or int(len(resultados)-1) <0:
                ID = input(f'Inserte un número entre 0-{len(resultados)-1}:')
        else:
            print(consulta_id(tabla, resultados[0]))
            ID = resultados[0]
    if algo == 'proveedor':
        ID_proveedor = ID
    else:
        ID_pieza = ID

cantidad = input(f'Indique cantidad de piezas:')
while cantidad.isnumeric() != True or int(cantidad) <= 0:
    cantidad = input(f'Inserte una cantidad:')

Suministros Industriales López
Tornillo hexagonal


In [17]:
ID_proveedor

6